---
# Jupyter notebook Bench_Grid_HiRep starts
---

## Importing the packages and checking if they are on system.
---

In [1074]:
# ----------------------------------------------------------------------------
# [Python-3.12] basic import for system check
# ----------------------------------------------------------------------------
import os
import sys
import time
import requests
import multiprocessing
import multiprocess
from collections import defaultdict
from functools import cache
from pathlib import Path
from glob import glob
# ----------------------------------------------------------------------------
# [Data-Plotting]
# ----------------------------------------------------------------------------
try:
    import matplotlib
    MATPLOTLIB_AVAILABLE = True
except (ImportError, NameError, AttributeError, OSError):
    print(" Python package matplotlib is not installed on your system, verify or install\n")
    MATPLOTLIB_AVAILABLE = False
try:
    import tqdm
    TQDM_AVAILABLE = True
except (ImportError, NameError, AttributeError, OSError):
    print(" Python package tqdm is not installed on your system, verify or install\n")
    TQDM_AVAILABLE = False
try:
    import pandas
    PANDAS_AVAILABLE = True
except (ImportError, NameError, AttributeError, OSError):
    print(" Python package pandas is not installed on your system, verify or install\n")
    PANDAS_AVAILABLE = False
try:
    import seaborn
    SEABORN_AVAILABLE = True
except (ImportError, NameError, AttributeError, OSError):
    print(" Python package seaborn is not installed on your system, verify or install\n")
    SEABORN_AVAILABLE = False

## Checking and printing check result
---

In [1075]:
# ----------------------------------------------------------------------------
# [Import-checks]
# ----------------------------------------------------------------------------
print("\n")
print("MATPLOTLIB_AVAILABLE --- installed --->: ", MATPLOTLIB_AVAILABLE)
print("TQDM_AVAILABLE       --- installed --->: ", TQDM_AVAILABLE)
print("PANDAS_AVAILABLE     --- installed --->: ", PANDAS_AVAILABLE)
print("SEABORN_AVAILABLE    --- installed --->: ", SEABORN_AVAILABLE)
print("\n")



MATPLOTLIB_AVAILABLE --- installed --->:  True
TQDM_AVAILABLE       --- installed --->:  True
PANDAS_AVAILABLE     --- installed --->:  True
SEABORN_AVAILABLE    --- installed --->:  True




## Setting up the main path structure and appending to system path.
---

In [1076]:
# ----------------------------------------------------------------------------
# [Path]
# ----------------------------------------------------------------------------
current_path = str(Path(sys.path[0]) / "..")
if current_path not in sys.path: sys.path.append(current_path)

APP_ROOT          = os.path.join(os.getcwd(), '..','..','..','..')
DATA_PATH         = os.path.join('E:','LatticeRuns','Clusters')
PROJECTNAME       = ""
POOL_COMPONENTDIR = ""
SOFTWARE          = "N/A"
SQL_DIR           = 'SQLFiles_sql'
APP_DATA_PATH     = "N/A"

DATAPROCINTERCOM  = ""
TBLECNTS_DIR      = ""
SQL_FULLPATH_DIR  = os.path.join(os.getcwd())
# ----------------------------------------------------------------------------
# [SystemPath-Appens]
# ----------------------------------------------------------------------------
sys.path.append(APP_ROOT)
sys.path.append(APP_DATA_PATH)
sys.path.append(DATA_PATH)
sys.path.append(os.path.join(APP_ROOT, '.'))
sys.path.append(os.path.join(APP_ROOT, '.','src','PythonCodes'))
sys.path.append(os.path.join(APP_ROOT, '.','src','PythonCodes','utils'))

print("Current Path  --->: ", current_path)
print("APP_ROOT      --->: ", APP_ROOT)
print("DATA_PATH     --->: ", DATA_PATH)
print("APP_DATA_PATH --->: ", APP_DATA_PATH)

Current Path  --->:  C:\cygwin64\home\Frederic\SwanSea\SourceCodes\Bench_Grid_HiRep\src\PythonCodes\Jupyter-NoteBooks\LLR-Bench_Grid_HiRep_Notebooks\..
APP_ROOT      --->:  C:\cygwin64\home\Frederic\SwanSea\SourceCodes\Bench_Grid_HiRep\src\PythonCodes\Jupyter-NoteBooks\LLR-Bench_Grid_HiRep_Notebooks\..\..\..\..
DATA_PATH     --->:  E:LatticeRuns\Clusters
APP_DATA_PATH --->:  N/A


## Importing the application objects.
---

In [1077]:
# ----------------------------------------------------------------------------
# [Application]
# ----------------------------------------------------------------------------
#Application imports
import src.PythonCodes.DataManage_common
import src.PythonCodes.utils.messageHandler
import src.PythonCodes.utils.Command_line
import src.PythonCodes.DataManage_header

## Instantiating the main objects
---

In [1078]:
version = src.PythonCodes.DataManage_common.DataManage_version()
c = src.PythonCodes.DataManage_common.DataManage_common()
rc = c.get_RC_SUCCESS()
# Getting the log file
logfile = c.getLogfileName()  #getting the name of the global log file
m = src.PythonCodes.utils.messageHandler.messageHandler(logfile = logfile)
# printing the header of Application
src.PythonCodes.DataManage_header.print_Bench_Grid_header(common=c, messageHandler=m)


 == [ ************************************************************************************] ==
 == [ *                    python code to run the Benchmarking suite.                    *] ==
 == [ *                                                                                  *] ==
 == [ * XXXXXX                          XX                XXXX             X        XX   *] ==
 == [ * X    X                          X               X    X                      X    *] ==
 == [ * X    X                          X              X                            X    *] ==
 == [ * X    X  XXXXX  XX XX    XXXXX   X XX           X       XXX XX   XXX     XXXXX    *] ==
 == [ * XXXXX  X     X  XX  X  X     X  XX  X          X         XX  X    X    X    X    *] ==
 == [ * X    X XXXXXXX  X   X  X        X   X  XXXXXXX X   XXX   X        X    X    X    *] ==
 == [ * X    X X        X   X  X        X   X          X     X   X        X    X    X    *] ==
 == [ * X    X X     X  X   X  X     X  X   X    

## Setting the path structure into the main objects
---

In [1079]:
c.setApp_root(APP_ROOT)
c.setData_path(DATA_PATH)
c.setProjectName(PROJECTNAME)
c.setPool_componentdir(POOL_COMPONENTDIR)
c.setSoftware(SOFTWARE)
c.setDataProcInterCom(DATAPROCINTERCOM)
c.setJSon_TableCounts_Dir(TBLECNTS_DIR)
c.setSql_dir(SQL_DIR)
c.setSql_fullPath_dir(SQL_FULLPATH_DIR)

## Starting the program
---

In [1080]:
# --------------------------------------------------------------------------
# [Main-code]
# --------------------------------------------------------------------------
m.printMesgStr("This is the main program      :", c.getCyan(), "Bench_Grid_HiRep.py")

 == [This is the main program      : Bench_Grid_HiRep.py] ==


## Main functions
---

### Reinitialising_Paths_and_object_content

In [1081]:
def Reinitialising_Paths_and_object_content(c, m, data_path, b_action, sim_sz):
    __func__= sys._getframe().f_code.co_name
    rc = c.get_RC_SUCCESS()
    m.printMesgStr("Getting target file list      :", c.getGreen(), __func__)
    # ----------------------------------------------------------------------
    c.setData_path(data_path)
    c.setTarget_File("target.txt")

    target_file_default = c.getTarget_File()
    m.printMesgStr("Default target file           :", c.getMagenta(), target_file_default)

    msg_analysis = c.getTarget_File().split(".txt")[0] + c.undr_scr + \
                    b_action      + c.undr_scr                      + \
                    sim_sz        + c.undr_scr                      + \
                    "batch_files" + c.txt_ext

    c.setTarget_File(str(msg_analysis))
    m.printMesgStr("Target file for analysis      :", c.getMagenta(), c.getTarget_File())

    c.setTargetdir( os.path.join(c.getData_path(), c.getTarget_File()))
    m.printMesgStr("Full Path target file         :", c.getCyan(), c.getTargetdir())

    if Path(c.getTargetdir()).is_file():
        m.printMesgAddStr("[Check]: target file       --->: ", c.getGreen(), "Exists")
    # ----------------------------------------------------------------------
    return rc
# [end-function]
# --------------------------------------------------------------------------

### getTarget_file_lst

In [1082]:
def getTarget_file_lst(c, m, target_file):
    __func__= sys._getframe().f_code.co_name
    rc = c.get_RC_SUCCESS()
    m.printMesgStr("Getting target file list      :", c.getGreen(), __func__)
    target_file_lst = []
    target_file_dir = []
    try:
        with open(target_file) as file:
            cnt = 0
            for line in file:
                target_file_lst.append(os.path.basename(line).strip())
                target_file_dir.append(os.path.dirname(os.path.realpath(line)).strip())
                cnt += 1
            # [end-For-Loop]
            m.printMesgAddStr("Number of files in target_file : ", c.getYellow(), str(cnt))
        # [end-with]
    except IOError:
        m.printMesgAddStr(" Filename          : ",
                                   c.getCyan(), target_file)
        m.printMesgAddStr("                   : ",
                                   c.getRed(), "cannot be found check"
                                                    " if file exist")
        #exit(c.get_RC_FAIL())
    # [end-try-catch]
    return rc, target_file_lst, target_file_dir
# [end-function]
# --------------------------------------------------------------------------

### getTaget_file_cluster_lst

In [1083]:
def getTarget_file_cluster_lst(c, m, target_file_lst):
    __func__= sys._getframe().f_code.co_name
    rc = c.get_RC_SUCCESS()
    m.printMesgStr("Getting target file list      :", c.getGreen(), __func__)
    target_file_cluster_lst = []
    for i in range(len(target_file_lst[:])):
        msg = (os.path.join(c.getData_path(), batch_action, simulation_size, str(target_file_lst[i].split(".sh")[0]), target_file_lst[i])).strip()
        if Path(msg).is_file():
            m.printMesgAddStr("[Check]: target file       --->: ", c.getGreen(), c.getMagenta()+ msg + c.getGreen() + " ---> Exists")
            # Now get the output file to analise and put it into a list
            cluster_out_file = msg.split(".sh")[0]+".out"
            # Extracting cluster files that has been benched
            if Path(cluster_out_file).is_file():
                m.printMesgAddStr("[Check]: Cluster file      --->: ", c.getGreen(), c.getYellow()+ cluster_out_file + c.getGreen() + " ---> Exists")
                target_file_cluster_lst.append(cluster_out_file)
            # [end-if]
        # [end-if]
    # [end-for-loop]
    return rc, target_file_cluster_lst
# [end-function]
# --------------------------------------------------------------------------

### readCluster_file_out

In [1084]:
def read_BKeeper_file_out(c, m, batch_action, simulation_size, target_file_cluster_lst):
    __func__= sys._getframe().f_code.co_name
    rc = c.get_RC_SUCCESS()
    m.printMesgStr("Getting target file list      :", c.getGreen(), __func__)
    end_key = "###############################################"
    bench_BKeeper_dict = {}
    start_key_rep_lst = [
        'Performing benchmark for SU(2), adjoint',
        'Performing benchmark for SU(2), fundamental',
        'Performing benchmark for SU(3), fundamental',
        'Performing benchmark for Sp(4), fundamental'
    ]
    start_BKeeper_key = "BKeeper"
    start_grid_key    = "Grid"
    # Starting the parsing of files over the start_key_lst
    # TODO: loop over the representation key
    ikey = 0
    cg_run_time_lst = []
    FlOps_GFlOps_lst = []
    Comms_MB_lst = []
    Memory_GB_lst = []
    mpi_distribution_lst = []
    nnodes_lst = []
    lattice_size_lst = []
    # Making sure that the list are empty before inserting anything
    cg_run_time_lst.clear()
    FlOps_GFlOps_lst.clear()
    Comms_MB_lst.clear()
    Memory_GB_lst.clear()
    mpi_distribution_lst.clear()
    nnodes_lst.clear()
    lattice_size_lst.clear()
    #
    for i in tqdm.tqdm(range(len(target_file_cluster_lst[:])), ncols=100, desc='bench_BKeeper_dict:'):
    #for i in range(len(target_file_cluster_lst[:])):
        cluster_file = open(target_file_cluster_lst[i])
        # Getting the mpi_distribution, lattice size and number of nodes
        ith_file = os.path.basename(target_file_cluster_lst[i].split('\n')[0]).split('.out')[0].split('Run_')[1].split(batch_action+'_')[1].split('_'+simulation_size)[0]
        split_string = ith_file.split('_')

        lines = cluster_file.readlines()
        database_file_len = len(lines)
        start_key_rep_ith = start_key_rep_lst[ikey]
        appending = False
        cnt = 0
        for j in range(database_file_len):
            if start_key_rep_ith in lines[j].split('\n')[0]:
                appending = True
            if end_key in lines[j].split('\n')[0]:
                appending = False
            elif appending:
                if start_BKeeper_key in lines[j].split('\n')[0]:
                    if "CG Run Time (s)" in  lines[j].split('\n')[0]:
                        key   = "CG Run Time (s)" #str(lines[j]).split(':')[0]
                        value = str(str(lines[j]).split(':')[4]).split('\n')[0]
                        cg_run_time_lst.append(float(value))

                        lattice_size_lst.append(str(split_string[0]).split('lat'  )[1])
                        mpi_distribution_lst.append(str(split_string[2]).split('mpi'  )[1])
                        nnodes_lst.append(str(split_string[1]).split('nodes')[1])

                    if "FlOp/S (GFlOp/s)" in  lines[j].split('\n')[0]:
                        key   = "FlOp/S (GFlOp/s)" #str(lines[j]).split(':')[0]
                        value = str(str(lines[j]).split(':')[4]).split('\n')[0]
                        FlOps_GFlOps_lst.append(float(value))
                    if "Comms  (MB)" in  lines[j].split('\n')[0]:
                        key   = "Comms" #str(lines[j]).split(':')[0]
                        value = str(str(lines[j]).split(':')[4]).split('\n')[0]
                        Comms_MB_lst.append(float(value))
                    if "Memory (GB)" in  lines[j].split('\n')[0]:
                        key   = "Memory (GB)" #str(lines[j]).split(':')[0]
                        value = str(str(lines[j]).split(':')[4]).split('\n')[0]
                        Memory_GB_lst.append(float(value))
            # [end-if]
        # [end-for-loop [j]]
    # [end-for-loop [i]]
    bench_BKeeper_dict["CG Run Time (s)"]  = cg_run_time_lst[:]
    bench_BKeeper_dict["FlOp/S (GFlOp/s)"] = FlOps_GFlOps_lst[:]
    bench_BKeeper_dict["Comms  (MB)"]      = Comms_MB_lst[:]
    bench_BKeeper_dict["Memory (GB)"]      = Memory_GB_lst[:]
    bench_BKeeper_dict["lattice"]          = lattice_size_lst[:]
    bench_BKeeper_dict["nodes"]            = nnodes_lst[:]
    bench_BKeeper_dict["mpi_distribution"] = mpi_distribution_lst[:]

    # creating a dictionary from the output data
    #print(" printing the dictionay ---->: ", bench_BKeeper_dict)
    dataframe = pandas.DataFrame.from_dict(bench_BKeeper_dict)

    return rc, dataframe
# [end-function]
# --------------------------------------------------------------------------

## Lumi
### [BKeeper_run_gpu : small]
---

In [1085]:
machine_name = "Lumi"
DATA_PATH         = os.path.join('E:','LatticeRuns','Clusters',machine_name,'LatticeRuns')

In [1086]:
batch_action = "BKeeper_run_gpu"
simulation_size="small"
rc = Reinitialising_Paths_and_object_content(c, m, DATA_PATH, batch_action, simulation_size)

 == [Getting target file list      : Reinitialising_Paths_and_object_content] ==
 == [Default target file           : target.txt] ==
 == [Target file for analysis      : target_BKeeper_run_gpu_small_batch_files.txt] ==
 == [Full Path target file         : E:LatticeRuns\Clusters\Lumi\LatticeRuns\target_BKeeper_run_gpu_small_batch_files.txt] ==
    [Check]: target file       --->: Exists


In [1087]:
# --------------------------------------------------------------------------
# Getting content in the target file
rc, target_file_lst_lumi, target_file_dir_lumi = getTarget_file_lst(c, m, c.getTargetdir())


 == [Getting target file list      : getTarget_file_lst] ==
    Number of files in target_file : 51


In [1088]:
# --------------------------------------------------------------------------
# Getting content in the target file
rc, target_file_cluster_lst_lumi = getTarget_file_cluster_lst(c, m, target_file_lst_lumi[:])

 == [Getting target file list      : getTarget_file_cluster_lst] ==
    [Check]: target file       --->: E:LatticeRuns\Clusters\Lumi\LatticeRuns\BKeeper_run_gpu\small\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi01-01-04-04_small\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi01-01-04-04_small.sh ---> Exists
    [Check]: Cluster file      --->: E:LatticeRuns\Clusters\Lumi\LatticeRuns\BKeeper_run_gpu\small\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi01-01-04-04_small\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi01-01-04-04_small.out ---> Exists
    [Check]: target file       --->: E:LatticeRuns\Clusters\Lumi\LatticeRuns\BKeeper_run_gpu\small\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi04-02-02-01_small\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi04-02-02-01_small.sh ---> Exists
    [Check]: Cluster file      --->: E:LatticeRuns\Clusters\Lumi\LatticeRuns\BKeeper_run_gpu\small\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi04-02-02-01_small\Run_BKeeper_run_gpu_lat24.24.24.32_

In [1089]:
# --------------------------------------------------------------------------
# [Data-Extraction]
# --------------------------------------------------------------------------
m.printMesgStr(   "Data extraction cluster out         : ", c.getGreen(), machine_name)
m.printMesgAddStr("Simulation size                     : ", c.getRed(), simulation_size)
m.printMesgAddStr("target_file_cluster_lst_lumi[:] --->: ", c.getYellow(), target_file_cluster_lst_lumi[:])

 == [Data extraction cluster out         :  Lumi] ==
    Simulation size                     : small
    target_file_cluster_lst_lumi[:] --->: ['E:LatticeRuns\\Clusters\\Lumi\\LatticeRuns\\BKeeper_run_gpu\\small\\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi01-01-04-04_small\\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi01-01-04-04_small.out', 'E:LatticeRuns\\Clusters\\Lumi\\LatticeRuns\\BKeeper_run_gpu\\small\\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi04-02-02-01_small\\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi04-02-02-01_small.out', 'E:LatticeRuns\\Clusters\\Lumi\\LatticeRuns\\BKeeper_run_gpu\\small\\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi01-08-02-01_small\\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi01-08-02-01_small.out', 'E:LatticeRuns\\Clusters\\Lumi\\LatticeRuns\\BKeeper_run_gpu\\small\\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi01-02-08-01_small\\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi01-02-08-01_small.out', 'E:LatticeRuns\\Clusters\\Lumi\\

In [1090]:
# --------------------------------------------------------------------------
# Reading in the inout file
rc, dataFrame_BKeeper_small_lumi = read_BKeeper_file_out(c, m, batch_action, simulation_size, target_file_cluster_lst_lumi[:])

 == [Getting target file list      : read_BKeeper_file_out] ==


bench_BKeeper_dict:: 100%|████████████████████████████████████████| 51/51 [00:00<00:00, 1819.13it/s]


In [1091]:
# --------------------------------------------------------------------------
# Reading in the inout file
dataFrame_BKeeper_small_lumi

,CG Run Time (s),FlOp/S (GFlOp/s),Comms (MB),Memory (GB),lattice,nodes,mpi_distribution
0,0.144327,1399.27649,46.006272,64.928076,24.24.24.32,002,01-08-02-01
1,0.110589,1826.16154,368.050176,69.133746,24.24.24.32,001,04-01-01-02
2,0.145297,1389.93495,184.025088,69.231542,24.24.24.32,002,04-02-01-02
3,0.136605,1478.37472,46.006272,65.441333,24.24.24.32,002,01-04-02-02
4,0.109194,1849.49153,368.050176,68.547166,24.24.24.32,001,04-01-02-01
5,0.138894,1454.01082,46.006272,68.520878,24.24.24.32,002,01-02-01-08
6,0.106646,1893.67982,92.012544,68.203114,24.24.24.32,001,01-01-01-08
7,0.107995,1870.02526,368.050176,68.547166,24.24.24.32,001,04-02-01-01
8,0.123481,1635.50164,184.025088,66.020360,24.24.24.32,001,02-04-01-01
9,0.114157,1769.08449,92.012544,66.296729,24.24.24.32,001,01-02-01-04


## Vega
### [BKeeper_run_gpu : small]
---

In [1092]:
machine_name = "Vega"
DATA_PATH         = os.path.join('E:','LatticeRuns','Clusters',machine_name,'LatticeRuns')

In [1093]:
# Reinitialising the paths and object content.
batch_action = "BKeeper_run_gpu"
simulation_size="small"
rc = Reinitialising_Paths_and_object_content(c, m, DATA_PATH, batch_action, simulation_size)

 == [Getting target file list      : Reinitialising_Paths_and_object_content] ==
 == [Default target file           : target.txt] ==
 == [Target file for analysis      : target_BKeeper_run_gpu_small_batch_files.txt] ==
 == [Full Path target file         : E:LatticeRuns\Clusters\Vega\LatticeRuns\target_BKeeper_run_gpu_small_batch_files.txt] ==
    [Check]: target file       --->: Exists


In [1094]:
# --------------------------------------------------------------------------
# Getting content in the target file
rc, target_file_lst_vega, target_file_dir_vega = getTarget_file_lst(c, m, c.getTargetdir())

 == [Getting target file list      : getTarget_file_lst] ==
    Number of files in target_file : 26


In [1095]:
# --------------------------------------------------------------------------
# Getting content in the target file
rc, target_file_cluster_lst_vega = getTarget_file_cluster_lst(c, m, target_file_lst_vega[:])

 == [Getting target file list      : getTarget_file_cluster_lst] ==
    [Check]: target file       --->: E:LatticeRuns\Clusters\Vega\LatticeRuns\BKeeper_run_gpu\small\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi01-01-04-02_small\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi01-01-04-02_small.sh ---> Exists
    [Check]: Cluster file      --->: E:LatticeRuns\Clusters\Vega\LatticeRuns\BKeeper_run_gpu\small\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi01-01-04-02_small\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi01-01-04-02_small.out ---> Exists
    [Check]: target file       --->: E:LatticeRuns\Clusters\Vega\LatticeRuns\BKeeper_run_gpu\small\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi04-02-01-01_small\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi04-02-01-01_small.sh ---> Exists
    [Check]: Cluster file      --->: E:LatticeRuns\Clusters\Vega\LatticeRuns\BKeeper_run_gpu\small\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi04-02-01-01_small\Run_BKeeper_run_gpu_lat24.24.24.32_

In [1096]:
# --------------------------------------------------------------------------
# [Data-Extraction]
# --------------------------------------------------------------------------
m.printMesgStr(   "Data extraction cluster out            : ", c.getGreen(), machine_name)
m.printMesgAddStr("Simulation size                        : ", c.getRed(), simulation_size)
m.printMesgAddStr("target_file_cluster_lst_vega[:]    --->: ", c.getYellow(), target_file_cluster_lst_vega[:])

 == [Data extraction cluster out            :  Vega] ==
    Simulation size                        : small
    target_file_cluster_lst_vega[:]    --->: ['E:LatticeRuns\\Clusters\\Vega\\LatticeRuns\\BKeeper_run_gpu\\small\\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi01-01-04-02_small\\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi01-01-04-02_small.out', 'E:LatticeRuns\\Clusters\\Vega\\LatticeRuns\\BKeeper_run_gpu\\small\\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi04-02-01-01_small\\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi04-02-01-01_small.out', 'E:LatticeRuns\\Clusters\\Vega\\LatticeRuns\\BKeeper_run_gpu\\small\\Run_BKeeper_run_gpu_lat24.24.24.32_nodes001_mpi04-01-01-01_small\\Run_BKeeper_run_gpu_lat24.24.24.32_nodes001_mpi04-01-01-01_small.out', 'E:LatticeRuns\\Clusters\\Vega\\LatticeRuns\\BKeeper_run_gpu\\small\\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi01-01-02-04_small\\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi01-01-02-04_small.out', 'E:LatticeRuns\\Cluster

In [1097]:
# --------------------------------------------------------------------------
# Reading in the inout file
rc, dataFrame_BKeeper_small_vega = read_BKeeper_file_out(c, m, batch_action, simulation_size, target_file_cluster_lst_vega[:])

 == [Getting target file list      : read_BKeeper_file_out] ==


bench_BKeeper_dict:: 100%|████████████████████████████████████████| 26/26 [00:00<00:00, 1296.48it/s]


In [1098]:
# --------------------------------------------------------------------------
# Reading in the inout file
dataFrame_BKeeper_small_vega

,CG Run Time (s),FlOp/S (GFlOp/s),Comms (MB),Memory (GB),lattice,nodes,mpi_distribution
0,0.495914,407.234678,368.050176,68.547166,24.24.24.32,002,04-02-01-01
1,0.377185,535.422613,736.100352,68.498268,24.24.24.32,001,04-01-01-01
2,0.209931,961.998839,92.012544,66.296729,24.24.24.32,002,01-01-02-04
3,0.228991,881.927143,92.012544,66.296729,24.24.24.32,002,01-02-01-04
4,0.257072,785.590723,92.012544,65.343537,24.24.24.32,002,01-04-01-02
5,0.271894,742.765115,184.025088,64.708059,24.24.24.32,001,01-04-01-01
6,0.288018,701.183184,184.025088,66.020360,24.24.24.32,002,02-04-01-01
7,0.236409,854.254188,92.012544,65.270214,24.24.24.32,002,01-02-02-02
8,0.296403,681.347282,368.050176,65.934801,24.24.24.32,001,02-01-02-01
9,0.237373,850.784960,368.050176,66.448058,24.24.24.32,001,02-01-01-02


## Leonardo
### [BKeeper_run_gpu : small]
---

In [1099]:
machine_name = "Leonardo"
DATA_PATH         = os.path.join('E:','LatticeRuns','Clusters',machine_name,'LatticeRuns')

In [1100]:
# Reinitialising the paths and object content.
batch_action = "BKeeper_run_gpu"
simulation_size="small"
rc = Reinitialising_Paths_and_object_content(c, m, DATA_PATH, batch_action, simulation_size)

 == [Getting target file list      : Reinitialising_Paths_and_object_content] ==
 == [Default target file           : target.txt] ==
 == [Target file for analysis      : target_BKeeper_run_gpu_small_batch_files.txt] ==
 == [Full Path target file         : E:LatticeRuns\Clusters\Leonardo\LatticeRuns\target_BKeeper_run_gpu_small_batch_files.txt] ==


In [1101]:
# --------------------------------------------------------------------------
# Getting content in the target file
rc, target_file_lst_leonardo, target_file_dir_leonardo = getTarget_file_lst(c, m, c.getTargetdir())

 == [Getting target file list      : getTarget_file_lst] ==
     Filename          : E:LatticeRuns\Clusters\Leonardo\LatticeRuns\target_BKeeper_run_gpu_small_batch_files.txt
                       : cannot be found check if file exist


In [1102]:
# --------------------------------------------------------------------------
# Getting content in the target file
rc, target_file_cluster_lst_leonardo = getTarget_file_cluster_lst(c, m, target_file_lst_leonardo[:])

 == [Getting target file list      : getTarget_file_cluster_lst] ==


In [1103]:
# --------------------------------------------------------------------------
# [Data-Extraction]
# --------------------------------------------------------------------------
m.printMesgStr(   "Data extraction cluster out             : ", c.getGreen(), machine_name)
m.printMesgAddStr("Simulation size                         : ", c.getRed(), simulation_size)
m.printMesgAddStr("target_file_cluster_lst_leonardo[:] --->: ", c.getYellow(), target_file_cluster_lst_leonardo[:])

 == [Data extraction cluster out             :  Leonardo] ==
    Simulation size                         : small
    target_file_cluster_lst_leonardo[:] --->: []


In [1104]:
# --------------------------------------------------------------------------
# Reading in the inout file
rc, dataFrame_BKeeper_small_leonardo = read_BKeeper_file_out(c, m, batch_action, simulation_size, target_file_cluster_lst_leonardo[:])

 == [Getting target file list      : read_BKeeper_file_out] ==


bench_BKeeper_dict:: 0it [00:00, ?it/s]


In [1105]:
# --------------------------------------------------------------------------
# Reading in the inout file
dataFrame_BKeeper_small_leonardo

,CG Run Time (s),FlOp/S (GFlOp/s),Comms (MB),Memory (GB),lattice,nodes,mpi_distribution
